<a href="https://colab.research.google.com/github/RehanaMudassar/1stAssignment/blob/main/Rag_proj_Rehana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.6 MB/s eta 0:00:00


In [15]:
import getpass
import os
import time
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)

In [17]:
import time
index_name = "mydata"
pc.create_index(
    name=index_name,
    dimension=768,
    # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(cloud="aws",region="us-east-1")
)
index=pc.Index(index_name)

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ["GOOGLE_API_KEY"]=userdata.get('GOOGLE_API_KEY')
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("I am learing langchain and pinecone collaboration!")
vector[:20]

[0.0370061956346035,
 0.0038398036267608404,
 -0.03125236555933952,
 -0.005780139472335577,
 0.025190329179167747,
 -0.009423061273992062,
 0.010473296046257019,
 -0.04979065805673599,
 0.04006427153944969,
 0.04326651617884636,
 0.026457209140062332,
 0.005406747572124004,
 -0.01541385892778635,
 -0.07913810759782791,
 -0.009121790528297424,
 -0.0419820174574852,
 0.03060910291969776,
 -0.03242318332195282,
 0.02388925850391388,
 0.010600889101624489]

In [19]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [20]:
#data save and data retrieve

In [22]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [23]:
len(documents)

10

In [24]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['1d0ff906-006b-4c9e-a60a-cd8899a9abb3',
 'b592537f-8423-4d4a-8476-b45985c99c2b',
 '5ceec551-b4ab-4169-adf5-e40bd23d428b',
 '682239f1-085b-4baf-82b8-23e6698d4a76',
 '6f03961b-3494-4696-9dec-253a96c4b42a',
 '51180879-3abb-401d-bd60-c6f7ee54fe2d',
 '40acdca2-ce26-4063-9421-7149502cbd37',
 '0ae693cb-2346-4d7c-bceb-025316bc8f71',
 'da2c219b-f0a8-4a36-8d04-2cc916bb2722',
 'ed364356-810f-4250-9b7f-54552f41fdaa']

In [25]:
from uuid import uuid4
uuid4()

UUID('986eb8c2-e166-42f5-99a8-9f1a6d41fba1')

In [26]:
#data Retrieval
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=3,
    filter={"source": "tweet"},
    )
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]


In [36]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [37]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='682239f1-085b-4baf-82b8-23e6698d4a76', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

In [38]:
def answer_to_user(query: str):
  results = vector_store.similarity_search(query, k=2)
  final_answer=model(results,query)
  return final_answer